In [3]:
from transformers import AutoModel, AutoTokenizer

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True).half().cuda()
model = model.eval()

tokenizer.save_pretrained("/content/drive/MyDrive/bernard/GLM_tokenizer")
model.save_pretrained("/content/drive/MyDrive/bernard/GLM_model")


A new version of the following files was downloaded from THUDM/chatglm-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from THUDM/chatglm-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from THUDM/chatglm-6b:
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from THUDM/chatglm-6b:
- modeling_chatglm.py
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -r /content/drive/MyDrive/bernard/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/EleutherAI/DeeperSpeed.git to /tmp/pip-install-6p6m54la/deepspeed_dfc9f843ccfd4bcdba539f01581112da
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/DeeperSpeed.git /tmp/pip-install-6p6m54la/deepspeed_dfc9f843ccfd4bcdba539f01581112da
  Resolved https://github.com/EleutherAI/DeeperSpeed.git to commit 5317ca6d0c02eeed3c0421e5d0b7956e020d5f85
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/EleutherAI/lm_dataformat.git (to revision 4eec05349977071bf67fc072290b95e31c8dd836) to /tmp/pip-req-build-hxispltp
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/lm_dataformat.git /tmp/pip-req-build-hxispltp
  Running command git rev-parse -q --verify 'sha^4eec05349977071bf67fc072290b95e31c8dd836'
  Running command git fetch -q https://github.com/EleutherAI/lm_dataformat.git 4eec05349

In [6]:
import re
import torch
from transformers import (AutoModel, AutoTokenizer, AutoModelForSequenceClassification, 
                          MarianMTModel, MarianTokenizer, DistilBertForSequenceClassification, 
                          DistilBertTokenizer, DistilBertConfig, AutoModelForSeq2SeqLM)

class DaevasAGI:
    def __init__(self):
        # Load models and tokenizers
        self.chat_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/bernard/GLM_tokenizer", trust_remote_code=True)
        self.chat_model = AutoModel.from_pretrained("/content/drive/MyDrive/bernard/GLM_model", trust_remote_code=True).half().cuda()
        self.chat_model.eval()

        self.sentiment_tokenizer = DistilBertTokenizer.from_pretrained("/content/drive/MyDrive/bernard/distilbert_sentiment_tokenizer")
        sentiment_config = DistilBertConfig.from_pretrained("/content/drive/MyDrive/bernard/distilbert_sentiment_config")
        self.sentiment_model = DistilBertForSequenceClassification(sentiment_config)
        self.sentiment_model.load_state_dict(torch.load("/content/drive/MyDrive/bernard/distilbert_sentiment.pt"))

        self.enghi_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/bernard/enghitokenizer")
        self.enghi_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/bernard/enghimodel")

        self.hieng_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/bernard/hiengtokenizer")
        self.hieng_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/bernard/hiengmodel")

    def predict(self, input_text, history=None):
        if history is None:
            history = []
        response, _ = self.chat_model.chat(self.chat_tokenizer, input_text, history)
        history.append({"role": "system", "content": "User: " + input_text})
        history.append({"role": "system", "content": "ChatGLM-6B: " + response})
        return history

    def summarize_text(self, text):
        prompt = f"Summarize: {text}"
        history = self.predict(prompt)
        summary = history[-1]["content"].replace("ChatGLM-6B: ", "")
        return summary

    def analyze_sentiment(self, text):
        inputs = self.sentiment_tokenizer.encode(text, return_tensors="pt")
        outputs = self.sentiment_model(inputs)
        sentiment = torch.argmax(outputs.logits, dim=1).item()
        return "positive" if sentiment == 1 else "negative"

    def translate_text_between_languages(self, text, source_language, target_language):
        if source_language == 'en' and target_language == 'hi':
            tokenizer = self.enghi_tokenizer
            model = self.enghi_model
        elif source_language == 'hi' and target_language == 'en':
            tokenizer = self.hieng_tokenizer
            model = self.hieng_model
        else:
            raise ValueError("Invalid language pair")

        batch = tokenizer([text], return_tensors="pt")
        generated_ids = model.generate(**batch)
        translation = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return translation

    def chatbot(self, prompt):
        prompt_lower = prompt.lower()

        if "?" in prompt:
            history = self.predict(prompt)
            response = history[-1]["content"].replace("ChatGLM-6B: ", "")

        elif "summarize" in prompt_lower:
            text_to_summarize = prompt.replace("summarize", "").strip()
            response = self.summarize_text(text_to_summarize)

        elif "write code" in prompt_lower:
            history = self.predict(prompt)
            response = history[-1]["content"].replace("ChatGLM-6B: ", "")

        elif "translate to hindi" in prompt_lower:
            text_to_translate = re.sub(r"translate to hindi\W+", "", prompt, flags=re.IGNORECASE).strip()
            response = self.translate_text_between_languages(text_to_translate, 'en', 'hi')

        elif "translate to english" in prompt_lower:
            text_to_translate = re.sub(r"translate to english\W+", "", prompt, flags=re.IGNORECASE).strip()
            response = self.translate_text_between_languages(text_to_translate, 'hi', 'en')

        elif "sentiment" in prompt_lower:
            text_to_analyze = prompt.replace("sentiment", "").strip()
            response = self.analyze_sentiment(text_to_analyze)

        else:
            history = self.predict(prompt)
            response = history[-1]["content"].replace("ChatGLM-6B: ", "")

        return response



In [7]:
if __name__ == "__main__":
    daevas = DaevasAGI()

    prompts = [
    "What is the capital of Vatican city?",
    "Summarize : We’ve designed the first universal framework and methodology for connecting a global semantic context for any data item with the computational cost to produce it - a design pattern that encodes symbolic information and compute cost into unique IDs, removing the need for cloud platform based measurement and analytics approaches for intelligent control systems. Instead, systems architects can rely on a game theoretic approach to network design, aggregating information from competing data service providers. This ensures that dynamic, resilient systems can be created today to withstand the extreme environments of network fragmentation in 2030.",
    "write code to convert my app.py to flask application. here is my python script : import re import torch from transformers import (AutoModel, AutoTokenizer, AutoModelForSequenceClassification, MarianMTModel, MarianTokenizer, DistilBertForSequenceClassification class DaevasAGI: def __init__(self): self.chat_tokenizer = AutoTokenizer.from_pretrained(/content/drive/MyDrive/bernard/chatglmmodel_tokenizer, trust_remote_code=True) self.chat_model = AutoModel.from_pretrained(/content/drive/MyDrive/bernard/chatglmmodel, trust_remote_code=True).half().cuda() self.chat_model.eval()  ",
    "translate to hindi: Hello, how are you",
    "translate to english: मेरा नाम रोहित है और मैं बर्लिन में रहता हूँ",
    "Sentiment: When it come to chicks, I never be the one to save,Keep on throwing ones, I'm with Mustard and we talkin' Lambo' talk (Skrrt, skrrt). Virgil dropped some Louis, I want everything he made and I'm just tryna relax. They always come off cool but in the end they come out wack (Wack). Her boyfriend getting mad, I told her he could take you back"
]


    for prompt in prompts:
        response = daevas.chatbot(prompt)
        print(f"User: {prompt}\nDaevasAGI: {response}\n")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

User: What is the capital of Vatican city?
DaevasAGI: The capital of Vatican City is St. Peter's Basilica, which is the site of the City of Rome's government and administrative buildings.

User: Summarize in less than 10 words: We’ve designed the first universal framework and methodology for connecting a global semantic context for any data item with the computational cost to produce it - a design pattern that encodes symbolic information and compute cost into unique IDs, removing the need for cloud platform based measurement and analytics approaches for intelligent control systems. Instead, systems architects can rely on a game theoretic approach to network design, aggregating information from competing data service providers. This ensures that dynamic, resilient systems can be created today to withstand the extreme environments of network fragmentation in 2030.
DaevasAGI: We designed a framework that encodes symbolic information and computational cost into unique IDs, removing the ne

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1344: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


User: translate to hindi: Hello, how are you
DaevasAGI: हैलो, तुम कैसे हो

User: translate to english: मेरा नाम रोहित है और मैं बर्लिन में रहता हूँ
DaevasAGI: My name is seconds and I live in Berlin.

User: Sentiment: When it come to chicks, I never be the one to save,Keep on throwing ones, I'm with Mustard and we talkin' Lambo' talk (Skrrt, skrrt). Virgil dropped some Louis, I want everything he made and I'm just tryna relax. They always come off cool but in the end they come out wack (Wack). Her boyfriend getting mad, I told her he could take you back
DaevasAGI: positive

